In [2]:
import pandas as pd
import yaml
import omero
from omero.gateway import BlitzGateway



In [3]:
data_path = '190225-JPB-info_data.xlsx'
df = pd.read_excel(data_path, sheet_name='data-GT-raw', header=[0, 1, 2])

# build fancy col names
col_names = []
for tup in df.columns.values:
  col_name = []
  for v in tup:
    if 'Unnamed:' not in str(v):
      col_name.append(str(v).strip()) #.replace(' ','-'))
  col_name=' '.join(col_name)
  # col_name='_'.join(col_name)
  col_names.append(col_name)
# merge multi columns into single string
df.columns = df.columns.map(str)
df.columns = col_names

# make numeric columns boolean
df[df.select_dtypes(include=['int64', 'float64']).columns] = df.select_dtypes(include=['int64', 'float64']).astype(bool)
df.set_index('Image', inplace=True)
df.head()

,Nature Substrat Homogene lisse,Nature Substrat Homogene stratifie,Nature Substrat Granuleux,Nature Substrat Fibrine fibrilles de fibrine,Nature Substrat Fibrine faisceaux fibrine,Nature Substrat Magma Erythrocytaire,Bacteries? nombre 0,Bacteries? nombre moins de 10,Bacteries? nombre plus de 10,Bacteries? nombre dans toute l'image,Bacteries? organisation isolees,Bacteries? organisation aggregees,Autres cellules? plaquettes,Autres cellules? blancs,Autres cellules? erythrocytes,Autres cellules? Calcifications
Image,,,,,,,,,,,,,,,,
060722-Nabila-JP-Valves-WholeMount-SAureus-pat02-15,False,False,True,False,False,False,False,False,True,False,True,True,False,False,False,False
060722-Nabila-JP-Valves-WholeMount-SAureus-pat02-29,False,False,True,False,True,False,False,True,False,False,True,False,False,False,True,False
060722-Nabila-JP-Valves-WholeMount-SAureus-pat03-06,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
060722-Nabila-JP-Valves-WholeMount-SAureus-pat04-1-07,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False
070220-pat06-WholeMountValve-32,True,False,False,False,False,False,False,False,True,False,True,True,True,False,False,True


In [3]:
fnames = df.index.values
tags = df.columns.values # will be the tags in the dataset metadata
print('index sample:'); print('\t'+'\n\t'.join(fnames[:5]))
print('columns:'); print('\t'+'\n\t'.join(tags))
nidx = 25; tidx=2
print('sample value:'); print('\t', f'{fnames[nidx]} -> {tags[tidx]} : ', df.loc[fnames[nidx],tags[tidx]])

index sample:
	060722-Nabila-JP-Valves-WholeMount-SAureus-pat02-15
	060722-Nabila-JP-Valves-WholeMount-SAureus-pat02-29
	060722-Nabila-JP-Valves-WholeMount-SAureus-pat03-06
	060722-Nabila-JP-Valves-WholeMount-SAureus-pat04-1-07
	070220-pat06-WholeMountValve-32
columns:
	Nature Substrat Homogene lisse
	Nature Substrat Homogene stratifie
	Nature Substrat Granuleux
	Nature Substrat Fibrine fibrilles de fibrine
	Nature Substrat Fibrine faisceaux fibrine
	Nature Substrat Magma Erythrocytaire
	Bacteries? nombre 0
	Bacteries? nombre moins de 10
	Bacteries? nombre plus de 10
	Bacteries? nombre dans toute l'image
	Bacteries? organisation isolees
	Bacteries? organisation aggregees
	Autres cellules? plaquettes
	Autres cellules? blancs
	Autres cellules? erythrocytes
	Autres cellules? Calcifications
sample value:
	 200721-JPB-SU5000-ValveWholeMount-pat16-A-11 -> Nature Substrat Granuleux :  True


In [ ]:
# OMERO connection details
hostname = 'localhost'
port = 4064
with open('secret.yaml', 'r') as file:
    credentials = yaml.safe_load(file)
username = credentials.get('USER1')
password = credentials.get('PASSWORD1')
dataset_id = 201 # all-GT-data

conn = None
try:
    # Connect to OMERO
    conn = BlitzGateway(username, password, host=hostname, port=port)
    conn.connect()

    # Get the dataset
    dataset = conn.getObject('Dataset', dataset_id)
    if not dataset:
        print(f"Dataset with ID {dataset_id} not found.")
        exit()

    # Show dataset info
    print(f'User: {conn.getUser().getFullName()}')
    print(f'Dataset group: {dataset.getDetails().getGroup().getName()}')
    project = dataset.getParent()
    if project:
        print(f'Dataset project: {project.getName()}')
    else:
        print('Dataset is not associated with any project.')
    print(f'Dataset name: {dataset.getName()}')

    # fill metadata
  

    for image in dataset.listChildren():
        iname = image.getName()
        print(f"Processing Image: {iname}")
       
        # make dictionary with metadata 
        key_value_dict = df.loc[iname.split('.')[0]].to_dict()
        
        # convert dict to list compatible with omero-py
        key_value_pairs = []
        for k,v in key_value_dict.items():
            key_value_pairs.append([k,str(v)])
        map_ann = omero.gateway.MapAnnotationWrapper(conn)
        
        # Use 'client' namespace to allow editing in Insight & web
        namespace = omero.constants.metadata.NSCLIENTMAPANNOTATION 
        map_ann.setNs(namespace) 
        map_ann.setValue(key_value_pairs)
        map_ann.save()

        #delete previous annotations
        to_delete = []
        # Optionally to filter by namespace
        for ann in image.listAnnotations(ns=namespace):
            to_delete.append(ann.id)
        if len(to_delete) != 0:
            conn.deleteObjects('Annotation', to_delete, wait=True)

        # add new annotations
        image.linkAnnotation(map_ann)

        break

except omero.SecurityViolation as sve:
    print(f"Authentication error: {sve}")
except omero.ClientError as ce:
    print(f"OMERO client error: {ce}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection
    if conn and conn.isConnected():
        conn.close()

User: Home User
Dataset group: system
Dataset project: semendo_sandbox
Dataset name: data-GT-raw
Processing Image: 060722-Nabila-JP-Valves-WholeMount-SAureus-pat05-25.tif


KeyboardInterrupt: 